In [2]:
# Install required libraries
!pip install -q transformers tqdm scikit-learn pillow ijson pandas gradio

In [3]:
# Import libraries

import os
import json
import ijson
import zipfile
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from transformers import AutoTokenizer, AutoModel

# Define Configuration

# 
import torch
import torch.nn as nn

# Detect GPUs
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"GPUs available: {num_gpus}")

    if num_gpus > 1:
        device = torch.device("cuda")
        multi_gpu = True
        print("Using MULTI-GPU (DataParallel).")
    else:
        device = torch.device("cuda:0")
        multi_gpu = False
        print("Using single GPU.")
else:
    device = torch.device("cpu")
    multi_gpu = False
    print("Using CPU.")

# ---------------------------
# Correct Path Configuration
# ---------------------------

# Path to the extracted image folder
PROJECT_ROOT = "/kaggle/input/mineeee2/recipe1m_tiny_100k"

# Images are inside recipe1m_tiny_50k/recipe1m_tiny_50k
FINAL_IMG_DIR = "/kaggle/input/mineeee2/recipe1m_tiny_100k"

# JSON files are directly under /mineee
LAYER1_JSON = "/kaggle/input/mineee/layer1.json"
LAYER2_JSON = "/kaggle/input/mineee/layer2.json"

IMAGES_DIR = FINAL_IMG_DIR  # For consistency
print("Configuration complete.")

# os.makedirs(IMAGES_DIR, exist_ok=True)

print("Configuration complete.")

GPUs available: 2
Using MULTI-GPU (DataParallel).
Configuration complete.
Configuration complete.


# **Data Preparation**

In [4]:
def unzip_data(zip_path, extract_to):

    # Check if we already unzipped (saves time on re-runs)
    if len(os.listdir(extract_to)) > 100:
        print(f"Folder {extract_to} already has files. Skipping unzip.")
        return

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Filter to extract only JPGs if zip is messy
        for file in tqdm(zip_ref.namelist(), desc="Extracting"):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                zip_ref.extract(file, extract_to)

    print("Extraction complete.")

# Run the extraction
unzip_data("/kaggle/input/mineeee2/recipe1m_tiny_100k", IMAGES_DIR)

# Handle nested folders (if zip created a subfolder)
contents = os.listdir(IMAGES_DIR)
# If the zip created a folder like "recipe1m_12k/" inside "images/", detect it
if len(contents) == 1 and os.path.isdir(os.path.join(IMAGES_DIR, contents[0])):
    FINAL_IMG_DIR = os.path.join(IMAGES_DIR, contents[0])
else:
    FINAL_IMG_DIR = IMAGES_DIR

print(f"Images are located in: {FINAL_IMG_DIR}")
print(f"Image Count: {len([f for f in os.listdir(FINAL_IMG_DIR) if f.endswith('.jpg')])}")

Folder /kaggle/input/mineeee2/recipe1m_tiny_100k already has files. Skipping unzip.
Images are located in: /kaggle/input/mineeee2/recipe1m_tiny_100k
Image Count: 100000


Matching Images to Text

In [5]:
# Get set of available image filenames on disk
available_images = set(os.listdir(FINAL_IMG_DIR))
print(f"Found {len(available_images)} images on disk.")

# 1. Map Image Filename -> Recipe ID (using layer2.json)
matched_recipe_ids = set()
id_to_img = {}

print("Streaming layer2+.json to match images")
with open(LAYER2_JSON, 'rb') as f:

    for entry in ijson.items(f, 'item'):
        rid = entry['id']
        for img in entry.get('images', []):
            fname = img['id']
            # If this image exists in our 12k folder
            if fname in available_images:
                matched_recipe_ids.add(rid)
                id_to_img[rid] = fname
                break

print(f"Matched {len(matched_recipe_ids)} Recipe IDs.")

# 2. Extract Text for those IDs (using layer1.json)
final_dataset = []

print("Streaming layer1.json to extract text")
with open(LAYER1_JSON, 'rb') as f:
    
    for entry in ijson.items(f, 'item'):
        rid = entry['id']
        if rid in matched_recipe_ids:
            final_dataset.append({
                "recipe_id": rid,
                "image_filename": id_to_img[rid],
                "title": entry.get("title", "Untitled"),
                "ingredients": [x['text'] for x in entry.get("ingredients", [])],
                "instructions": [x['text'] for x in entry.get("instructions", [])]
            })

print(f"Final Dataset Size: {len(final_dataset)} pairs.")

Found 100000 images on disk.
Streaming layer2+.json to match images
Matched 48982 Recipe IDs.
Streaming layer1.json to extract text
Final Dataset Size: 48982 pairs.


In [6]:
# Save cleaned dataset in Kaggle Working directory (writable)
OUTPUT_PATH = "/kaggle/working/dataset_100k_clean.json"

with open(OUTPUT_PATH, "w") as f:
    json.dump(final_dataset, f)

print(f"Dataset saved to: {OUTPUT_PATH}")

Dataset saved to: /kaggle/working/dataset_100k_clean.json


# **Data Preprocessing**

In [7]:
# A. AUGMENTATION
# We use augmentation for training to force the model to learn features, not pixels.
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),        # Resize slightly larger than target
    transforms.RandomCrop(224),           # Randomly crop the 224x224 area
    transforms.RandomHorizontalFlip(),    # Flip left/right (food looks same flipped)
    transforms.RandomRotation(15),        # Rotate +/- 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # Vary lighting
    transforms.ToTensor(),                # Convert to Tensor
    transforms.Normalize(                 # Normalize to ImageNet standards
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# For Validation, resize and normalize (No randomness)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


# B. DATASET CLASS

class RecipeDataset(Dataset):
    def __init__(self, data_list, img_dir, transform, tokenizer, max_len=128):
        self.data = data_list
        self.img_dir = img_dir
        self.transform = transform
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # 1. Image
        img_path = os.path.join(self.img_dir, item['image_filename'])
        try:
            image = Image.open(img_path).convert("RGB")
            image = self.transform(image)
        except:
            # Fallback for corrupted images
            image = torch.zeros(3, 224, 224)

        # 2. Text (Concatenate Title + Ingredients + Instructions)
        # We join them to give BERT maximum context
        text_input = f"{item['title']} [SEP] " + \
                     " ".join(item['ingredients']) + \
                     " [SEP] " + \
                     " ".join(item['instructions'])

        encoded = self.tokenizer(
            text_input,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            "image": image,
            "input_ids": encoded['input_ids'].squeeze(0),
            "attention_mask": encoded['attention_mask'].squeeze(0),
            "recipe_id": item['recipe_id'] # Keep ID for lookup later
        }

print("Transforms and Dataset class defined.")

Transforms and Dataset class defined.


In [8]:
#Dataset Split and Loaders
DATASET_PATH = "/kaggle/working/dataset_100k_clean.json"

with open(DATASET_PATH, "r") as f:
    full_data = json.load(f)

print(f"Loaded {len(full_data)} (image, text) pairs from {DATASET_PATH}")


# 2. 3-WAY SPLIT: Train (80%), Validation (10%), Test (10%)

from sklearn.model_selection import train_test_split

# To Ensure reproducibility
RANDOM_SEED = 42

# Shuffle entire dataset once
np.random.seed(RANDOM_SEED)
np.random.shuffle(full_data)

# Train / Temp split (80% / 20%)
train_list, temp_list = train_test_split(
    full_data,
    test_size=0.20,
    random_state=RANDOM_SEED
)

# Temp → Val/Test split (50% / 50% of the 20%)
val_list, test_list = train_test_split(
    temp_list,
    test_size=0.50,
    random_state=RANDOM_SEED
)

print("Dataset Splits:")
print(f" Train      : {len(train_list)} ({len(train_list)/len(full_data)*100:.1f}%)")
print(f" Validation : {len(val_list)} ({len(val_list)/len(full_data)*100:.1f}%)")
print(f" Test       : {len(test_list)} ({len(test_list)/len(full_data)*100:.1f}%)")



# 3. TOKENIZER (DistilBERT)

tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased",
    model_max_length=128,
    padding="max_length",
    truncation=True
)

print("Tokenizer initialized.")



# 4. DATASETS

train_dataset = RecipeDataset(
    train_list, FINAL_IMG_DIR, train_transform, tokenizer
)
val_dataset = RecipeDataset(
    val_list, FINAL_IMG_DIR, val_transform, tokenizer
)
test_dataset = RecipeDataset(
    test_list, FINAL_IMG_DIR, val_transform, tokenizer
)

print("Datasets constructed.")


# 5. DATA LOADERS

BATCH_SIZE = 64

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=2, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=2, pin_memory=True
)

test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=2, pin_memory=True
)

print(f"DataLoaders ready (batch_size={BATCH_SIZE}).")

Loaded 48982 (image, text) pairs from /kaggle/working/dataset_100k_clean.json
Dataset Splits:
 Train      : 39185 (80.0%)
 Validation : 4898 (10.0%)
 Test       : 4899 (10.0%)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer initialized.
Datasets constructed.
DataLoaders ready (batch_size=64).


# **Model Architecture and Training**

In [9]:
# ---------------------------------------------------------
# DUAL ENCODER
# ---------------------------------------------------------
class DualEncoder(nn.Module):
    def __init__(self, embed_dim=256, freeze_cnn=True, freeze_text_epochs=2):
        super().__init__()

        self.freeze_text_epochs = freeze_text_epochs

        # ---------------------------------------------------------
        # IMAGE ENCODER (RESNET50)
        # ---------------------------------------------------------
        self.cnn = models.resnet50(weights="IMAGENET1K_V2")
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, embed_dim)

        # Freeze CNN initially
        if freeze_cnn:
            for p in self.cnn.parameters():
                p.requires_grad = False
            for p in self.cnn.fc.parameters():
                p.requires_grad = True

        # ---------------------------------------------------------
        # TEXT ENCODER (DISTILBERT)
        # ---------------------------------------------------------
        self.text_encoder = AutoModel.from_pretrained("distilbert-base-uncased")
        self.text_proj = nn.Linear(768, embed_dim)

        # Freeze text encoder initially
        for p in self.text_encoder.parameters():
            p.requires_grad = False

    def unfreeze_text_encoder(self):
        print("Unfreezing Text Encoder…")
        for p in self.text_encoder.parameters():
            p.requires_grad = True

    def forward(self, images=None, input_ids=None, attention_mask=None):
        img_features = None
        txt_features = None

        if images is not None:
            img_features = self.cnn(images)
            img_features = F.normalize(img_features, p=2, dim=1)

        if input_ids is not None:
            txt_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
            txt_features = txt_out.last_hidden_state[:, 0, :]
            txt_features = self.text_proj(txt_features)
            txt_features = F.normalize(txt_features, p=2, dim=1)

        return img_features, txt_features


In [10]:
# ---------------------------------------------------------
# TRAINING FUNCTION
# ---------------------------------------------------------
def train_model(
    model,
    train_loader,
    val_loader,
    epochs=20,
    lr=1e-4,
    patience=4,
    unfreeze_cnn_epoch=3,
    temperature=0.07,
    save_path=f"{PROJECT_ROOT}/best_model.pt"
):

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=epochs,
        eta_min=lr * 0.1
    )

    best_val_loss = float("inf")
    patience_counter = 0

    def contrastive_loss(img_emb, txt_emb, temperature=temperature):
        logits = (img_emb @ txt_emb.T) / temperature
        logits = torch.clamp(logits, -50, 50)

        labels = torch.arange(len(logits)).to(device)
        loss_i = nn.CrossEntropyLoss()(logits, labels)
        loss_t = nn.CrossEntropyLoss()(logits.T, labels)

        return (loss_i + loss_t) / 2

    print("Starting Training...")

    for epoch in range(epochs):

        # Unfreeze text encoder AFTER warmup
        if epoch == model.freeze_text_epochs:
            model.unfreeze_text_encoder()

        # Unfreeze CNN later
        if epoch == unfreeze_cnn_epoch:
            print(f"Unfreezing CNN at epoch {epoch}")
            for p in model.cnn.parameters():
                p.requires_grad = True

        model.train()
        total_train_loss = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in loop:
            images = batch['image'].to(device)
            input_ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)

            img_emb, txt_emb = model(images, input_ids, mask)
            loss = contrastive_loss(img_emb, txt_emb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        avg_train_loss = total_train_loss / len(train_loader)

        # ------------------------ VALIDATION ------------------------
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                input_ids = batch['input_ids'].to(device)
                mask = batch['attention_mask'].to(device)

                img_emb, txt_emb = model(images, input_ids, mask)
                total_val_loss += contrastive_loss(img_emb, txt_emb).item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        # ---------------------- EARLY STOPPING ----------------------
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            print("Saved NEW BEST MODEL!")
        else:
            patience_counter += 1
            print(f"No improvement ({patience_counter}/{patience})")

            if patience_counter >= patience:
                print("Early Stopping Triggered.")
                break

        scheduler.step()

    print("Training Finished.")
    print(f"Best model saved at: {save_path}")


In [11]:
# TRAINING CALL
model = DualEncoder(embed_dim=256, freeze_cnn=True).to(device)

train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=20,
    lr=1e-4,
    patience=4,
    unfreeze_cnn_epoch=3,
    save_path="/kaggle/working/best_model.pt"
)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]
2025-11-29 17:49:28.049562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764438568.236157      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764438568.291638      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...


Epoch 15/20
Train Loss: 0.0721 | Val Loss: 1.0078
No improvement (2/4)


Epoch 16/20: 100%|██████████| 613/613 [13:07<00:00,  1.28s/it, loss=0.0116]



Epoch 16/20
Train Loss: 0.0655 | Val Loss: 1.0153
No improvement (3/4)


Epoch 17/20: 100%|██████████| 613/613 [13:06<00:00,  1.28s/it, loss=0.00978]



Epoch 17/20
Train Loss: 0.0578 | Val Loss: 1.0206
No improvement (4/4)
Early Stopping Triggered.
Training Finished.
Best model saved at: /kaggle/working/best_model.pt


# **Evaluation**

In [27]:
# Load the Best Saved Model
model = DualEncoder(embed_dim=256, freeze_cnn=False).to(device)
model.load_state_dict(torch.load("/kaggle/working/best_model.pt", map_location=device))
model.eval()
print("Best model loaded for testing.")


# 1. COMPUTE EMBEDDINGS FOR TEST SET
image_embeddings = []
text_embeddings = []
test_recipe_ids = []

print("Computing embeddings for Test Set")

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)

        # Get embeddings
        img_emb, txt_emb = model(images, input_ids, mask)

        # Store them on CPU to save GPU memory
        image_embeddings.append(img_emb.cpu())
        text_embeddings.append(txt_emb.cpu())
        test_recipe_ids.extend(batch['recipe_id'])

# Concatenate all batches
image_embeddings = torch.cat(image_embeddings)
text_embeddings = torch.cat(text_embeddings)

print(f"Test Embeddings Shape: {image_embeddings.shape}")

# ---------------------------------------------------------
# 2. CALCULATE COSINE SIMILARITY MATRIX
# ---------------------------------------------------------
# Normalize
image_embeddings = F.normalize(image_embeddings, p=2, dim=1)
text_embeddings = F.normalize(text_embeddings, p=2, dim=1)

# Similarity Matrix: [N_test x N_test]
# row i, col j = similarity between Image_i and Text_j
similarity_matrix = image_embeddings @ text_embeddings.T

# similarity_matrix shape: [N_images, N_texts]


# IMAGE → TEXT RETRIEVAL


def recall_at_k_i2t(similarity_matrix, k=1):
    N = similarity_matrix.shape[0]
    correct = 0

    for i in range(N):
        sims = similarity_matrix[i]
        topk_idx = torch.topk(sims, k).indices.tolist()
        if i in topk_idx:
            correct += 1

    return correct / N


def median_rank_i2t(similarity_matrix):
    N = similarity_matrix.shape[0]
    ranks = []

    for i in range(N):
        sims = similarity_matrix[i]
        sorted_idx = torch.argsort(sims, descending=True)
        rank = (sorted_idx == i).nonzero(as_tuple=False).item() + 1
        ranks.append(rank)

    return float(np.median(ranks))



# TEXT → IMAGE RETRIEVAL


def recall_at_k_t2i(similarity_matrix, k=1):
    sim_t2i = similarity_matrix.T
    N = sim_t2i.shape[0]
    correct = 0

    for i in range(N):
        sims = sim_t2i[i]
        topk_idx = torch.topk(sims, k).indices.tolist()
        if i in topk_idx:
            correct += 1

    return correct / N


def median_rank_t2i(similarity_matrix):
    sim_t2i = similarity_matrix.T
    N = sim_t2i.shape[0]
    ranks = []

    for i in range(N):
        sims = sim_t2i[i]
        sorted_idx = torch.argsort(sims, descending=True)
        rank = (sorted_idx == i).nonzero(as_tuple=False).item() + 1
        ranks.append(rank)

    return float(np.median(ranks))



# PRINT RESULTS

print("\n")
print("IMAGE → TEXT RETRIEVAL RESULTS")

for k in [1, 5, 10]:
    print(f"Recall@{k}: {recall_at_k_i2t(similarity_matrix, k):.4f}")

print(f"Median Rank (MedR): {median_rank_i2t(similarity_matrix):.2f}")


print("\n")
print("TEXT → IMAGE RETRIEVAL RESULTS")

for k in [1, 5, 10]:
    print(f"Recall@{k}: {recall_at_k_t2i(similarity_matrix, k):.4f}")

print(f"Median Rank (MedR): {median_rank_t2i(similarity_matrix):.2f}")


Best model loaded for testing.
Computing embeddings for Test Set


Testing: 100%|██████████| 77/77 [00:51<00:00,  1.50it/s]


Test Embeddings Shape: torch.Size([4899, 256])


IMAGE → TEXT RETRIEVAL RESULTS
Recall@1: 0.1664
Recall@5: 0.3882
Recall@10: 0.4997
Median Rank (MedR): 11.00


TEXT → IMAGE RETRIEVAL RESULTS
Recall@1: 0.1678
Recall@5: 0.3911
Recall@10: 0.4979
Median Rank (MedR): 11.00


# **Nutrition Optimization (Dietary constraint + Goals)**

In [13]:
import pandas as pd
import zipfile
import os
import re

NUTRITION_EXTRACT_DIR = "/kaggle/input/nutrition-data"

if not os.path.exists(NUTRITION_EXTRACT_DIR):
    print(f"Unzipping {NUTRITION_ZIP_PATH}")
    try:
        with zipfile.ZipFile(NUTRITION_ZIP_PATH, 'r') as z:
            z.extractall(NUTRITION_EXTRACT_DIR)
        print("Extraction complete.")
    except Exception as e:
        print(f"Error unzipping: {e}")
else:
    print("Folder already exists, skipping unzip.")


# 2. FIND ALL DATA CSVs
data_csv_files = []
for root, dirs, files in os.walk(NUTRITION_EXTRACT_DIR):
    for file in files:
        if file.endswith(".csv") and "FOOD-DATA" in file and "METADATA" not in file:
            data_csv_files.append(os.path.join(root, file))

if not data_csv_files:
    raise FileNotFoundError("No suitable CSV files found!")

print(f"Found {len(data_csv_files)} data files. Merging...")

# 3. LOAD & MERGE KAGGLE NUTRITION DATA

def get_val(row, possible_cols):
    """Helper to find the first valid column that exists"""
    for col in possible_cols:
        if col in row:
            return float(row[col])
    return 0.0

def load_all_nutrition_files(file_list):
    lookup = {}

    for csv_path in file_list:
        try:
            df = pd.read_csv(csv_path)
            # Normalize columns
            df.columns = [c.strip().lower().replace(" ", "_").replace(".", "") for c in df.columns]

            #Iterate rows
            for _, row in df.iterrows():
                # Get raw name
                raw_name = str(row.get('food', '')).lower()
                # Clean name: "Chicken, raw" -> "chicken raw"
                clean_name = ''.join(c for c in raw_name if c.isalnum() or c.isspace()).strip()

                if not clean_name: continue

                # Extract Macros using flexible column names
                macros = {
                    'calories': get_val(row, ['caloric_value', 'calories', 'energy_kcal']),
                    'protein':  get_val(row, ['protein', 'protein_g']),
                    'fat':      get_val(row, ['fat', 'total_lipid_fat']),
                    'carbs':    get_val(row, ['carbohydrates', 'carbohydrate_by_difference']),
                    'sugar':    get_val(row, ['sugars', 'sugars_total'])
                }

                # STRATEGY: SAVE MULTIPLE KEYS
                # 1. Save the full name: "chicken raw"
                lookup[clean_name] = macros

                # Save the FIRST word: "chicken"
                first_word = clean_name.split()[0]
                if len(first_word) > 2 and first_word not in lookup:
                     lookup[first_word] = macros

        except Exception as e:
            print(f"Error reading {csv_path}: {e}")

    return lookup

nutrition_lookup = load_all_nutrition_files(data_csv_files)
print(f"Successfully loaded {len(nutrition_lookup)} lookup keys.")

# 4. HEURISTIC INGREDIENT PARSER
#INGREDIENT PARSER


def calculate_approx_macros(ingredients_list):
    """
    1. Multi-word matching (e.g., 'chicken breast', 'olive oil')
    2. Whole-word matching using regex boundaries
    3. Fallback to single-word matching
    4. Avoids false positives (egg in eggplant)
    """

    total = {'calories': 0.0, 'protein': 0.0, 'fat': 0.0, 'carbs': 0.0, 'sugar': 0.0}
    matched_count = 0

    # Pre-clean all nutrition keys for quick search
    nutrition_keys = list(nutrition_lookup.keys())

    for ingredient in ingredients_list:
        ingredient_clean = ingredient.lower()

        # 1. MULTI-WORD MATCHING
        match_found = False

        for food_key in nutrition_keys:
            # multi-word exact phrase match
            pattern = r"\b" + re.escape(food_key) + r"\b"
            if re.search(pattern, ingredient_clean):
                macros = nutrition_lookup[food_key]
                for k in total:
                    total[k] += macros[k]
                matched_count += 1
                match_found = True
                break

        if match_found:
            continue  # skip token matching if phrase match worked

        # 2. SINGLE-WORD TOKEN MATCHING (fallback)
        
        # extract only alphabetical tokens
        tokens = re.findall(r"[a-zA-Z]+", ingredient_clean)

        for token in tokens:
            if len(token) <= 2: 
                continue  # skip useless small words

            # whole-word match
            if token in nutrition_lookup:
                macros = nutrition_lookup[token]
                for k in total:
                    total[k] += macros[k]
                matched_count += 1
                match_found = True
                break

    # 3. AVERAGE OVER MATCHED INGREDIENTS
    if matched_count > 0:
        for k in total:
            total[k] = round(total[k] / matched_count, 2)
    else:
        # if nothing matched, return zeros
        total = {k: 0.0 for k in total}

    return total, matched_count

Folder already exists, skipping unzip.
Found 5 data files. Merging...
Successfully loaded 3112 lookup keys.


Nutrition

In [50]:
# DIETARY FILTER
import re

def check_dietary_compliance(ingredients_list, diet_filter, title=""):
    """
    Dietary filter:
    - Case-insensitive
    - Whole-word boundaries (\bX\b)
    - Multi-word detection
    - Checks BOTH ingredients + title
    """

    diet = diet_filter.lower().strip()

    # Combine ingredients and title into a unified text blob
    text_blob = (title + " " + " ".join(ingredients_list)).lower()

    # FOOD CATEGORY LISTS
    MEAT = [
        "chicken", "chicken breast", "ground chicken",
        "beef", "ground beef", "steak", "veal",
        "mutton", "lamb", "goat", "buff"
        "pork", "bacon", "ham", "pepperoni", "salami", "Pork"
        "sausage", "turkey", "ground chuck", "chuck", "hen", "tenderloin", "sirloin"
    ]

    FISH = [
        "fish", "seafood", "mixed seafood", "frozen seafood",
        "seafood mix", "seafood medley",
        "salmon", "tuna", "shrimp", "prawn", "anchovy", "cod",
        "crab", "lobster", "scallop", "mussel", "clam", "oyster",
        "squid", "octopus", "mackerel", "trout", "snapper", "scallops", "clams"
    ]

    DAIRY = [
        "milk", "cheese", "butter", "yogurt",
        "cream", "parmesan", "paneer", "ghee", "whey"
    ]

    EGGS = ["egg", "egg yolk", "eggs"]

    HONEY = ["honey"]

    PORK = ["pork", "bacon", "ham", "salami", "pepperoni", "lard", "chorizo", "Pork"]

    ALCOHOL = ["wine", "vodka", "rum", "beer", "whiskey", "bourbon", "liqueur"]

    GLUTEN = ["wheat", "flour", "bread", "noodles", "pasta", "spaghetti", "barley", "rye", "soy sauce"]

    # Helpers for whole-word matching
    def contains(words):
        for w in words:
            pattern = r"\b" + re.escape(w) + r"\b"
            if re.search(pattern, text_blob):
                return True
        return False

    # -----------------------------
    # APPLY DIET RULES
    # -----------------------------
    
    # Vegan: no meat, fish, dairy, eggs, honey
    if diet == "vegan":
        if contains(MEAT+FISH+DAIRY+EGGS+HONEY):
            return False

    # Vegetarian: no meat, no fish
    if diet == "vegetarian":
        if contains(MEAT+FISH+PORK+EGGS):
            return False

    # Pescatarian: no meat, fish allowed
    if diet == "pescatarian":
        if contains(MEAT):
            return False

    # Halal: no pork, no alcohol
    if diet == "halal":
        if contains(PORK+ALCOHOL):
            return False

    # Gluten free
    if diet == "gluten free":
        if contains(GLUTEN):
            return False
    return True


# **Testing**

In [16]:
#BUILD FULL DATABASE INDEX (Train + Val + Test)

# 1. Prepare Data
all_recipe_ids = [item['recipe_id'] for item in full_data]
full_db_embeddings = []

# 2. Create a DataLoader for the Full Dataset (Text Only)
# We can reuse the RecipeDataset class but we only need text
full_dataset = RecipeDataset(full_data, FINAL_IMG_DIR, val_transform, tokenizer)
full_loader = DataLoader(full_dataset, batch_size=64, shuffle=False, num_workers=2)

# 3. Compute Embeddings
model.eval()
with torch.no_grad():
    for batch in tqdm(full_loader, desc="Indexing Full DB"):
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)

        # Get Text Embeddings from the Model
        # We manually call the text branch since we don't have images here
        txt_out = model.text_encoder(input_ids=input_ids, attention_mask=mask)
        txt_feat = txt_out.last_hidden_state[:, 0, :]
        txt_feat = model.text_proj(txt_feat)
        txt_feat = F.normalize(txt_feat, p=2, dim=1)

        full_db_embeddings.append(txt_feat.cpu())

# 4. Concatenate and move to device once, making it a global variable
global db_emb
db_emb = torch.cat(full_db_embeddings).to(device)
print(f"Full Database Indexed. Shape: {db_emb.shape}")

Indexing Full DB: 100%|██████████| 766/766 [07:34<00:00,  1.69it/s]


Full Database Indexed. Shape: torch.Size([48982, 256])


In [37]:
import torch.nn.functional as F
from PIL import Image as PILImage

#SETUP
if 'id_to_entry' not in globals():
    id_to_entry = {item['recipe_id']: item for item in full_data}

# RETRIEVAL FUNCTION
def retrieve_recipe_from_image(image_path, topk=50):
    if topk is None: topk = 50
    try:
        img = PILImage.open(image_path).convert("RGB")
        img_t = val_transform(img).unsqueeze(0).to(device)
    except:
        return []

    with torch.no_grad():
        img_emb, _ = model(images=img_t)
        img_emb = F.normalize(img_emb, p=2, dim=1)

    sims = (img_emb @ db_emb.T).squeeze(0)

    max_k = min(int(topk), len(all_recipe_ids))
    top_indices = torch.topk(sims, k=max_k).indices.tolist()

    results = []
    for idx in top_indices:
        rid = all_recipe_ids[idx]
        entry = id_to_entry.get(rid)
        if entry:
            results.append({
                "recipe_id": rid,
                "similarity": sims[idx].item(),
                "image_filename": entry['image_filename'],
                "title": entry['title']
            })
    return results

In [38]:
# NORMALIZED NUTRITION SCORING

# Convert nutrition lookup (all individual foods) into a DataFrame
all_macros = pd.DataFrame(nutrition_lookup.values())

# Fix column names
all_macros = all_macros.rename(columns={
    "sugar": "sugar"  # ensure consistent naming
})

max_cal = all_macros["calories"].max()
max_pro = all_macros["protein"].max()
max_fat = all_macros["fat"].max()
max_carb = all_macros["carbs"].max()
max_sug = all_macros["sugar"].max()

print("Max macro values computed.")

def nutrition_objective_score(macros, objective):
    if macros is None:
        return 0.0

    cal  = macros.get("calories", 0)
    pro  = macros.get("protein", 0)
    fat  = macros.get("fat", 0)
    carb = macros.get("carbs", 0)
    sug  = macros.get("sugar", 0)

    # Normalization
    cal_n  = cal  / (max_cal  + 1e-6)
    pro_n  = pro  / (max_pro  + 1e-6)
    fat_n  = fat  / (max_fat  + 1e-6)
    carb_n = carb / (max_carb + 1e-6)
    sug_n  = sug  / (max_sug  + 1e-6)

    # Objective scoring
    if objective == "high_protein":
        return pro_n

    if objective == "low_calorie":
        return 1.0 - cal_n

    if objective == "low_fat":
        return 1.0 - fat_n

    if objective == "low_carb":
        return 1.0 - carb_n

    if objective == "sugar_free":
        return 1.0 - sug_n

    if objective == "keto":
        return 0.7*(1-carb_n) + 0.3*(fat_n)

    return 0.0

Max macro values computed.


In [39]:
# RE-RANKING FUNCTION
def rerank_with_constraints(results, diet_type="none", objective="none", alpha=0.5):
    final_list = []
    
    # sanitize inputs
    diet_type = diet_type.lower()
    objective = objective.lower()

    for item in results:
        rid = item["recipe_id"]
        entry = id_to_entry.get(rid)
        if entry is None:
            continue

        # Compute macros
        macros, _ = calculate_approx_macros(entry['ingredients'])

        # Apply diet filter
        if diet_type != "none":
            if not check_dietary_compliance(entry['ingredients'], diet_type, entry['title']):
                continue

        # Nutrition score
        nut_score = nutrition_objective_score(macros, objective)

        # Visual similarity
        sim = float(item["similarity"])

        # Final score (balanced)
        final_score = (alpha * nut_score) + ((1 - alpha) * sim)

        enriched = {
            "title": entry.get("title", "Unknown"),
            "image_filename": entry.get("image_filename", ""),
            "ingredients": entry.get("ingredients", []),
            "instructions": entry.get("instructions", []),
            "final_score": final_score,
            "macros": macros,
            "similarity": sim
        }
        final_list.append(enriched)

    # Sort descending
    final_list.sort(key=lambda x: x["final_score"], reverse=True)
    return final_list


In [51]:
# TESTING ON IMAGE
import os

test_image_path = os.path.join(PROJECT_ROOT, "/kaggle/input/images-for-test/creamy-mushroom-pasta-recipe-8.jpg")
print(f"Query Image Path: {test_image_path}")

print("Step 1: Retrieving Top 10 visual matches...")
raw_results = retrieve_recipe_from_image(test_image_path, topk=200)

diet = "vegetarian"
goal = "low_calorie"

print(f"Step 2: Re-ranking with Diet='{diet}' and Goal='{goal}'...")

ranked_results = rerank_with_constraints(
    raw_results,
    diet_type=diet,
    objective=goal,
    alpha=0.7
)

print("\n" + "="*90)
print(f"TOP 10 RESULTS FOR '{goal.upper()}'")
print("="*90)

print(f"{'RANK':<5} | {'SCORE':<8} | {'CAL':<6} | {'PRO':<6} | {'FAT':<6} | {'CARB':<6} | {'SUG':<6} | TITLE")
print("-" * 90)

for i, r in enumerate(ranked_results[:10]):
    cal = r["macros"]["calories"]
    pro = r["macros"]["protein"]
    fat = r["macros"]["fat"]
    carb = r["macros"]["carbs"]
    sug = r["macros"]["sugar"]

    print(
        f"#{i+1:<4} | {r['final_score']:.4f} | "
        f"{cal:<6.1f} | {pro:<6.1f} | {fat:<6.1f} | {carb:<6.1f} | {sug:<6.1f} | "
        f"{r['title'][:40]}"
    )

print("="*90)
print("ℹNOTE: Final Score = α * NutritionScore + (1-α) * VisualSimilarity")


Query Image Path: /kaggle/input/images-for-test/creamy-mushroom-pasta-recipe-8.jpg
Step 1: Retrieving Top 10 visual matches...
Step 2: Re-ranking with Diet='vegetarian' and Goal='low_calorie'...
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done

TOP 10 RESULTS FOR 'LOW_CALORIE'
RANK  | SCORE    | CAL    | PRO    | FAT    | CARB   | SUG    | TITLE
------------------------------------------------------------------------------------------
#1    | 0.9303 | 59.1   | 1.0    | 2.4    | 7.3    | 0.5    | Linguine con Vongole
#2    | 0.9094 | 104.6  | 1.5    | 4.5    | 15.0   | 9.7    | Fast and Easy Fettuccine
#3    | 0.9074 | 71.9   | 1.6    | 3.0    | 9.7   

In [48]:
# Interactive UI-Based Testing

import gradio as gr
import torch.nn.functional as F
from PIL import Image as PILImage
import numpy as np
import os

# FORMATTER
def format_multi_result(recipes, topk=2):
    output_text = ""
    for i, r in enumerate(recipes[:topk]):
        m = r['macros']

        output_text += f"## Result {i+1}: {r['title']}\n\n"

        output_text += f"**Score:** {r['final_score']:.3f}\n\n"
        output_text += (
            f"**Estimated Nutrition:** {m['calories']} cal | "
            f"**{m['protein']}g Protein** | {m['fat']}g Fat | {m['carbs']}g Carbs\n\n"
        )

        output_text += "### 🥣 Ingredients\n"
        ing_formatted = "\n".join([f"- {ing}" for ing in r['ingredients'][:8]])
        output_text += ing_formatted + "\n"
        if len(r['ingredients']) > 8:
            output_text += f"- ... ({len(r['ingredients']) - 8} more)\n"

        output_text += "\n### 📝 Instructions\n"
        inst_formatted = "\n".join([f"{j+1}. {inst}" 
                                     for j, inst in enumerate(r['instructions'][:5])])
        output_text += inst_formatted + "\n"
        if len(r['instructions']) > 5:
            output_text += f"- ... ({len(r['instructions']) - 5} more steps)\n"

        output_text += "\n---\n\n"
    return output_text


# ---------------------------------------------------------
# MAIN GRADIO FUNCTION
# ---------------------------------------------------------
def gradio_retrieve(img, diet_type, objective, topk):
    if img is None:
        return "Please upload an image.", None

    img_path = "/content/query_image.jpg"
    img.save(img_path)

    raw = retrieve_recipe_from_image(img_path, topk=topk)

    final = rerank_with_constraints(
        raw,
        diet_type=diet_type.lower().strip(),   
        objective=objective.lower().strip(), 
        alpha=0.3
    )

    if not final:
        return "No recipes found.", None

    text_block = format_multi_result(final, topk=3)

    gallery_data = []
    for r in final[:3]:
        img_path = os.path.join(FINAL_IMG_DIR, r["image_filename"])
        if os.path.exists(img_path):
            gallery_data.append((PILImage.open(img_path), r["title"]))

    return text_block, gallery_data



# GRADIO UI

if __name__ == '__main__':

    diet_opts = ['none','vegan','vegetarian','pescatarian','halal','gluten free']

    goal_opts = ['none','high_protein','low_calorie','low_fat','low_carb',
                 'sugar_free','keto']

    image_input = gr.Image(type="pil", label="Upload an image")

    diet_dropdown = gr.Dropdown(choices=diet_opts, value='none',
                                label="Dietary Preference")

    objective_dropdown = gr.Dropdown(choices=goal_opts, value='none',
                                     label="Nutritional Goal")

    topk_slider = gr.Slider(minimum=5, maximum=200, value=50, step=5,
                             label="Retrieve Top-K Candidates")

    text_output = gr.Markdown(label="Recipe Details")
    gallery_output = gr.Gallery(label="Recipe Images")

    demo = gr.Interface(
        fn=gradio_retrieve,
        inputs=[image_input, diet_dropdown, objective_dropdown, topk_slider],
        outputs=[text_output, gallery_output],
        title="Recipe Retrieval (Image → Recipe + Nutrition + Diet Filters)",
        description="Upload a dish image and get matching recipes filtered by diet and nutritional goals."
    )

    demo.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://6b8305e1bad8ec5cba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6b8305e1bad8ec5cba.gradio.live
